In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,RepeatedKFold
from sklearn.linear_model import Ridge,Lasso, LassoCV,LinearRegression
from xgboost import XGBRegressor
from model_functions import label_encoding,training_model,nn_models
from sklearn.model_selection import train_test_split
dataset=pd.read_excel("dataset.xlsx")
a=pd.read_excel("sample2.xlsx")

[NbConvertApp] Converting notebook model_functions.ipynb to script
[NbConvertApp] Writing 2362 bytes to model_functions.py


In [2]:
dataset.head()

,Destination,Origin,To Area,Flight Date,day_name,flight_month,season,year,Flight Code,Days,...,Block1,Sold1,Left1,Occ.(%)1,Occ.,Netto,Netto Currency,Profit,prıce,day_convert
0,Turkey,Belgorod,Antalya,02.01.2020,Thursday,January,Winter,2020,WZ 4035.,4,...,220,151,69,69,95.45,177.0,EUR,61.73,238.73,2020-01-02
1,Turkey,Belgorod,Antalya,09.01.2020,Thursday,January,Winter,2020,WZ 4035.,4,...,220,220,0,100,0.00,174.0,EUR,NaN,174.00,2020-01-09
2,Turkey,Chelyabinsk,Antalya,02.01.2020,Thursday,January,Winter,2020,U6 1009,4,...,220,181,39,82,97.73,253.0,EUR,58.35,311.35,2020-01-02
3,Turkey,Chelyabinsk,Antalya,10.01.2020,Friday,January,Winter,2020,U6 1009,5,...,220,217,3,99,0.00,236.0,EUR,NaN,236.00,2020-01-10
4,Turkey,Chelyabinsk,Antalya,15.03.2020,Sunday,March,Spring,2020,WZ 4009,7,...,220,1,219,0,99.09,251.0,EUR,-50.69,200.31,2020-03-15


In [3]:
dataset.corr()

,year,Days,Block,Sold,Left,Occ.(%),Block1,Sold1,Left1,Occ.(%)1,Occ.,Netto,Profit,prıce
year,1.000000,-0.029299,-0.500965,-0.437583,-0.277592,0.202734,-0.500537,-0.436137,-0.241933,0.177326,0.203446,0.673736,0.554636,0.686331
Days,-0.029299,1.000000,0.019128,0.019513,0.002197,0.025203,0.017977,0.025472,-0.017529,0.026723,0.025176,-0.014492,0.003030,-0.004891
Block,-0.500965,0.019128,1.000000,0.948956,0.328061,-0.119312,0.998016,0.932570,0.314093,-0.069487,-0.119934,-0.549725,-0.403521,-0.523769
Sold,-0.437583,0.019513,0.948956,1.000000,0.013363,0.123732,0.947692,0.871648,0.335404,-0.088854,0.123050,-0.500339,-0.348561,-0.454279
Left,-0.277592,0.002197,0.328061,0.013363,1.000000,-0.748825,0.325558,0.345833,-0.008803,0.045833,-0.748755,-0.244215,-0.269738,-0.299880
Occ.(%),0.202734,0.025203,-0.119312,0.123732,-0.748825,1.000000,-0.116721,-0.137044,0.038051,-0.039430,0.999966,0.134688,0.270740,0.265150
Block1,-0.500537,0.017977,0.998016,0.947692,0.325558,-0.116721,1.000000,0.934785,0.313752,-0.068651,-0.117335,-0.549923,-0.403008,-0.523321
Sold1,-0.436137,0.025472,0.932570,0.871648,0.345833,-0.137044,0.934785,1.000000,-0.043987,0.182550,-0.137541,-0.497350,-0.360413,-0.471084
Left1,-0.241933,-0.017529,0.314093,0.335404,-0.008803,0.038051,0.313752,-0.043987,1.000000,-0.681047,0.037651,-0.217206,-0.169732,-0.212599
Occ.(%)1,0.177326,0.026723,-0.069487,-0.088854,0.045833,-0.039430,-0.068651,0.182550,-0.681047,1.000000,-0.039222,0.105084,0.156977,0.155612


In [4]:
dataset.describe()

,year,Days,Block,Sold,Left,Occ.(%),Block1,Sold1,Left1,Occ.(%)1,Occ.,Netto,Profit,prıce
count,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9376.000000,9265.000000,9376.000000
mean,2021.590444,4.075299,110.070286,100.846416,9.223869,93.901877,109.839484,100.654224,9.185260,93.139078,93.893706,357.159776,244.885018,599.145664
std,0.672453,2.013205,115.047927,108.690466,36.290315,18.139012,114.947229,109.248709,40.870404,20.796022,18.139970,131.830580,215.333670,303.057957
min,2020.000000,1.000000,1.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,-6.000000,0.000000,0.000000,50.000000,-555.910000,-99.210000
25%,2021.000000,2.000000,34.000000,30.000000,0.000000,100.000000,34.000000,30.000000,0.000000,100.000000,99.590000,258.000000,55.920000,346.937500
50%,2022.000000,4.000000,63.000000,62.000000,0.000000,100.000000,63.000000,62.000000,0.000000,100.000000,100.000000,360.245000,229.110000,620.515000
75%,2022.000000,6.000000,186.000000,163.000000,1.000000,100.000000,186.000000,163.000000,0.000000,100.000000,100.000000,405.700000,397.390000,799.690000
max,2022.000000,7.000000,492.000000,492.000000,478.000000,103.000000,492.000000,494.000000,478.000000,107.000000,102.940000,750.000000,2552.460000,3255.640000


In [5]:
dataset.corr().unstack().sort_values(ascending=False).drop_duplicates().head(50)

year      year        1.000000
Occ.      Occ.(%)     0.999966
Block     Block1      0.998016
          Sold        0.948956
Sold      Block1      0.947692
Block1    Sold1       0.934785
Block     Sold1       0.932570
Profit    prıce       0.926078
Sold1     Sold        0.871648
prıce     Netto       0.784713
          year        0.686331
Netto     year        0.673736
year      Profit      0.554636
Netto     Profit      0.494763
Left      Sold1       0.345833
Sold      Left1       0.335404
Left      Block       0.328061
          Block1      0.325558
Left1     Block       0.314093
          Block1      0.313752
Profit    Occ.        0.271347
Occ.(%)   Profit      0.270740
prıce     Occ.        0.265738
Occ.(%)   prıce       0.265150
Occ.      year        0.203446
year      Occ.(%)     0.202734
Occ.(%)1  Sold1       0.182550
          year        0.177326
          Profit      0.156977
          prıce       0.155612
Occ.      Netto       0.135200
Netto     Occ.(%)     0.134688
Sold    

In [6]:
corr_matrix = dataset.corr()
threshold = 0.3

In [7]:
high_corr = corr_matrix[corr_matrix >= threshold].stack().reset_index()
high_corr = high_corr[high_corr['level_0'] != high_corr['level_1']]
high_corr = high_corr.rename(columns={'level_0': 'Feature 1', 'level_1': 'Feature 2', 0: 'Correlation'})
high_corr = high_corr.sort_values(by='Correlation', ascending=False)

print(high_corr)

   Feature 1 Feature 2  Correlation
21   Occ.(%)      Occ.     0.999966
38      Occ.   Occ.(%)     0.999966
8      Block    Block1     0.998016
22    Block1     Block     0.998016
6      Block      Sold     0.948956
11      Sold     Block     0.948956
13      Sold    Block1     0.947692
23    Block1      Sold     0.947692
26    Block1     Sold1     0.934785
31     Sold1    Block1     0.934785
9      Block     Sold1     0.932570
28     Sold1     Block     0.932570
47    Profit     prıce     0.926078
50     prıce    Profit     0.926078
14      Sold     Sold1     0.871648
29     Sold1      Sold     0.871648
49     prıce     Netto     0.784713
43     Netto     prıce     0.784713
48     prıce      year     0.686331
3       year     prıce     0.686331
1       year     Netto     0.673736
40     Netto      year     0.673736
2       year    Profit     0.554636
44    Profit      year     0.554636
45    Profit     Netto     0.494763
42     Netto    Profit     0.494763
19      Left     Sold1     0

In [8]:
high_corr['Feature 1'] = high_corr['Feature 1'].astype(str)
high_corr['Feature 2'] = high_corr['Feature 2'].astype(str)
core_feature = high_corr['Feature 1'].unique().tolist()
core_feature += high_corr['Feature 2'].unique().tolist()

In [9]:
core_feature

['Occ.(%)',
 'Occ.',
 'Block',
 'Block1',
 'Sold',
 'Sold1',
 'Profit',
 'prıce',
 'Netto',
 'year',
 'Left',
 'Left1',
 'Occ.',
 'Occ.(%)',
 'Block1',
 'Block',
 'Sold',
 'Sold1',
 'prıce',
 'Profit',
 'Netto',
 'year',
 'Left',
 'Left1']

In [12]:
core_feature=np.unique(core_feature)

In [13]:
dataset.columns

Index(['Destination', 'Origin', 'To Area', 'Flight Date', 'day_name',
       'flight_month', 'season', 'year', 'Flight Code', 'Days',
       'Airline Company', 'dpt', 'Block', 'Sold', 'Left', 'Occ.(%)', 'dpt1',
       'Block1', 'Sold1', 'Left1', 'Occ.(%)1', 'Occ.', 'Netto',
       'Netto Currency', 'Profit', 'prıce', 'day_convert'],
      dtype='object')

In [14]:
new_dataset = dataset.loc[:, core_feature]
new_dataset

,Block,Block1,Left,Left1,Netto,Occ.,Occ.(%),Profit,Sold,Sold1,prıce,year
0,220,220,10,69,177.00,95.45,95,61.73,210,151,238.73,2020
1,220,220,220,0,174.00,0.00,0,NaN,0,220,174.00,2020
2,220,220,5,39,253.00,97.73,98,58.35,215,181,311.35,2020
3,220,220,220,3,236.00,0.00,0,NaN,0,217,236.00,2020
4,220,220,2,219,251.00,99.09,99,-50.69,218,1,200.31,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
9371,65,65,0,0,377.18,100.00,100,420.95,65,65,798.13,2022
9372,46,46,0,0,343.18,100.00,100,398.44,46,46,741.62,2022
9373,34,34,0,0,370.00,100.00,100,512.82,34,34,882.82,2022
9374,54,54,0,0,377.18,100.00,100,467.51,54,54,844.69,2022


In [15]:
new_dataset=new_dataset.loc[:,['year','Block','Sold','Left','Occ.(%)','Block1','Sold1',
                                       'Left1','Occ.','Netto','Profit','prıce']]

In [16]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9376 entries, 0 to 9375
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     9376 non-null   int64  
 1   Block    9376 non-null   int64  
 2   Sold     9376 non-null   int64  
 3   Left     9376 non-null   int64  
 4   Occ.(%)  9376 non-null   int64  
 5   Block1   9376 non-null   int64  
 6   Sold1    9376 non-null   int64  
 7   Left1    9376 non-null   int64  
 8   Occ.     9376 non-null   float64
 9   Netto    9376 non-null   float64
 10  Profit   9265 non-null   float64
 11  prıce    9376 non-null   float64
dtypes: float64(4), int64(8)
memory usage: 879.1 KB


In [17]:
new_dataset

,year,Block,Sold,Left,Occ.(%),Block1,Sold1,Left1,Occ.,Netto,Profit,prıce
0,2020,220,210,10,95,220,151,69,95.45,177.00,61.73,238.73
1,2020,220,0,220,0,220,220,0,0.00,174.00,NaN,174.00
2,2020,220,215,5,98,220,181,39,97.73,253.00,58.35,311.35
3,2020,220,0,220,0,220,217,3,0.00,236.00,NaN,236.00
4,2020,220,218,2,99,220,1,219,99.09,251.00,-50.69,200.31
...,...,...,...,...,...,...,...,...,...,...,...,...
9371,2022,65,65,0,100,65,65,0,100.00,377.18,420.95,798.13
9372,2022,46,46,0,100,46,46,0,100.00,343.18,398.44,741.62
9373,2022,34,34,0,100,34,34,0,100.00,370.00,512.82,882.82
9374,2022,54,54,0,100,54,54,0,100.00,377.18,467.51,844.69


In [39]:
new_dataset.columns

Index(['year', 'Block', 'Sold', 'Left', 'Occ.(%)', 'Block1', 'Sold1', 'Left1',
       'Occ.', 'Netto', 'Profit', 'prıce'],
      dtype='object')

In [18]:
label_encoding(dataset, a, 'Destination')
label_encoding(dataset, a, 'Origin')
label_encoding(dataset, a, 'To Area')
label_encoding(dataset, a, 'day_name')
label_encoding(dataset, a, 'flight_month')

label_encoding(dataset, a, 'season')
label_encoding(dataset, a, 'Netto Currency')
label_encoding(dataset, a, 'Flight Code')

label_encoding(dataset, a, 'Airline Company')
label_encoding(dataset, a, 'Flight Date')

label_encoding(dataset, a, 'dpt')
label_encoding(dataset, a, 'dpt1')

In [19]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Destination      11 non-null     int64  
 1   Origin           11 non-null     int64  
 2   To Area          11 non-null     int64  
 3   Flight Date      11 non-null     int64  
 4   day_name         11 non-null     int64  
 5   flight_month     11 non-null     int64  
 6   season           11 non-null     int64  
 7   year             11 non-null     int64  
 8   Flight Code      11 non-null     int64  
 9   Days             11 non-null     int64  
 10  Airline Company  11 non-null     int64  
 11  dpt              11 non-null     int64  
 12  Block            11 non-null     int64  
 13  Sold             11 non-null     int64  
 14  Left             11 non-null     int64  
 15  Occ.(%)          11 non-null     int64  
 16  dpt1             11 non-null     int64  
 17  Block1           1

In [20]:
new_dataset["Profit"]=new_dataset["Profit"].fillna(new_dataset["Profit"].median())

In [21]:
X = new_dataset.drop(['prıce'], axis=1)
y = new_dataset.iloc[:, -1:]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.33, random_state=42)

In [24]:
xgb = XGBRegressor()

param_grid = {
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1] ,
    "max_depth": range(3,21,3),
    "gamma": [i/10.0 for i in range(0,5)],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]
}

xgb_cv = RandomizedSearchCV(xgb, param_grid, n_iter = 100, refit='recall',
         scoring='neg_mean_squared_error', cv = 5, verbose=2, random_state=42, n_jobs = -1) 
xgb_cv.fit(X_train, y_train)
xgb_reg = XGBRegressor(**xgb_cv.best_params_ )

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [45]:
sample2 = a.loc[:,['year', 'Block', 'Sold', 'Left', 'Occ.(%)', 'Block1', 'Sold1', 'Left1',
       'Occ.', 'Netto', 'Profit']]
sample2

,year,Block,Sold,Left,Occ.(%),Block1,Sold1,Left1,Occ.,Netto,Profit
0,2021,220,144,76,65,220,212,8,65.45,149.00,-32.53
1,2022,189,134,55,71,189,189,0,70.90,246.00,-42.98
2,2022,58,58,0,100,58,58,0,100.00,475.82,562.33
3,2022,16,16,0,100,14,14,0,100.00,660.36,647.46
4,2021,163,163,0,100,163,163,0,100.00,356.36,318.20
5,2022,492,489,3,99,478,8,470,99.39,162.00,81.50
6,2022,475,473,2,100,474,0,474,99.58,150.00,153.87
7,2022,328,326,2,99,328,322,6,99.39,162.00,22.08
8,2022,119,114,5,96,110,82,28,95.80,200.00,75.78
9,2022,235,55,180,23,235,235,0,23.40,259.00,68.88


In [46]:
rf2 = RandomForestRegressor(n_estimators = 11, max_depth = 30,min_samples_split=5,random_state = 42)
DT_Regressor = tree.DecisionTreeRegressor()
ridge = Ridge(alpha=1.0)
lasso_cv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lr = LinearRegression()

In [47]:
models = { 'Random forest': rf2,'Decision tree': DT_Regressor,'Ridge regression': ridge,
          'Lasso regression': lasso_cv,'Linear regression': lr,'XGBOOST': xgb_reg}

In [48]:
model_outputs = {}
sample_predicted={}

for i, (key, model) in enumerate(models.items()):
    model, score, mse, rmse, mae = training_model(X_train, X_test, y_train, y_test, model=model)
    model_outputs[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
    sample_predicted[key]=model.predict(sample2)

R^squared: 0.9972743774907535
MSE:  248.90326143829625
RMSE:  15.776668261654494
MAE: 5.101360360308412
R^squared: 0.998386120004096
MSE:  147.37917418416805
RMSE:  12.13998246226773
MAE: 5.213373182552497
R^squared: 0.9939509300910401
MSE:  552.3997633203554
RMSE:  23.503186237622238
MAE: 7.43217273276021
R^squared: 0.9939384909281549
MSE:  553.5357050001727
RMSE:  23.52733952235511
MAE: 7.455243716383414
R^squared: 0.9939507489113729
MSE:  552.4163086086008
RMSE:  23.503538214673146
MAE: 7.421801515129494


C:\Users\90541\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


R^squared: 0.9985146530106427
MSE:  135.64156766550568
RMSE:  11.646525991277642
MAE: 5.144974270994734


In [49]:
model_outputs = pd.DataFrame(model_outputs)
model_outputs

,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
R-squared,0.997274,0.998386,0.993951,0.993938,0.993951,0.998515
MSE,248.903261,147.379174,552.399763,553.535705,552.416309,135.641568
RMSE,15.776668,12.139982,23.503186,23.527340,23.503538,11.646526
MAE,5.101360,5.213373,7.432173,7.455244,7.421802,5.144974


In [50]:
sample_predicted = pd.DataFrame(sample_predicted)

In [52]:
real_pred=pd.concat([a[['prıce']],sample_predicted], axis=1)
real_pred

,prıce,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
0,116.47,114.818515,113.84,97.743018,98.115516,97.864415,115.148003
1,203.02,189.777538,200.31,189.175599,189.720154,189.373304,197.581696
2,1038.15,1046.139410,1051.36,1035.601689,1035.169251,1035.635449,1038.028687
3,1307.82,1304.225297,1305.01,1303.945537,1303.331208,1303.961293,1303.974854
4,674.56,645.919634,671.63,674.672911,674.533879,674.655169,597.222046
5,243.50,243.460384,243.50,242.700485,245.655570,242.178885,261.004456
6,303.87,298.873838,303.87,302.582371,305.606310,301.979184,333.570221
7,184.08,183.313853,184.08,188.897722,188.958619,189.080536,172.432922
8,275.78,275.148063,275.78,276.926182,277.666584,277.014163,307.721222
9,327.88,314.025727,327.88,279.531873,280.144214,279.909633,308.804169


In [53]:
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.python.keras.layers import Dense, LSTM, RepeatVector, Flatten,SimpleRNN

In [54]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [55]:
print('Train shape', X_train_scaled.shape)
print('Test shape', X_test_scaled.shape)

Train shape (6281, 11)
Test shape (3095, 11)


In [56]:
# input [samples, time steps, features] for LSTM
X_train_items = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_items = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [57]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(500, 50), max_iter=2000, random_state=42)

In [58]:
modellstm = Sequential()
modellstm.add(LSTM(50, input_shape=(X_train_items.shape[1], 1)))
modellstm.add(Dense(1))
modellstm.compile(optimizer='adam', loss='mean_squared_error')

In [59]:
rnn = Sequential()
rnn.add(SimpleRNN(units=20, input_shape=(X_train_items.shape[1], X_train_items.shape[2])))
rnn.add(Dense(1))
rnn.compile(loss='mean_squared_error', optimizer='adam')

In [60]:
models = { 'MLP': mlp,'LSTM': modellstm,'RNN': rnn}

In [61]:
model_outputs_nn = {}
sample_predicted_nn={}

for i, (key, model) in enumerate(models.items()):
    if key == 'MLP':
        model, score, mse, rmse, mae=training_model(X_train_scaled, X_test_scaled, y_train, y_test ,model)
        model_outputs_nn[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_data = scaler.transform(sample2)
        sample_predicted_nn[key]=model.predict(sample_data)
    else:
        model, score, mse, rmse, mae =nn_models(X_train_scaled, X_test_scaled, y_train, y_test, model, epoch_num=200, batch_size=16)
        model_outputs_nn[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_predicted_nn[key]=model.predict(sample2)

R^squared: 0.9968270195984388
MSE:  289.755887966575
RMSE:  17.022217480885825
MAE: 4.49426715376861
Epoch 1/200
393/393 [==============================] - 3s 4ms/step - loss: 426460.2812
Epoch 2/200
393/393 [==============================] - 2s 4ms/step - loss: 401589.2812
Epoch 3/200
393/393 [==============================] - 2s 4ms/step - loss: 380029.0938
Epoch 4/200
393/393 [==============================] - 2s 5ms/step - loss: 359757.2812
Epoch 5/200
393/393 [==============================] - 2s 4ms/step - loss: 340552.8438
Epoch 6/200
393/393 [==============================] - 2s 6ms/step - loss: 322286.0312
Epoch 7/200
393/393 [==============================] - 2s 5ms/step - loss: 304916.8750
Epoch 8/200
393/393 [==============================] - 2s 4ms/step - loss: 288399.9062
Epoch 9/200
393/393 [==============================] - 2s 4ms/step - loss: 272699.5000
Epoch 10/200
393/393 [==============================] - 2s 4ms/step - loss: 257759.2656
Epoch 11/200
393/393 [======

393/393 [==============================] - 2s 5ms/step - loss: 1276.0596
Epoch 188/200
393/393 [==============================] - 2s 5ms/step - loss: 1252.9425
Epoch 189/200
393/393 [==============================] - 2s 4ms/step - loss: 1246.8264
Epoch 190/200
393/393 [==============================] - 2s 4ms/step - loss: 1237.6992
Epoch 191/200
393/393 [==============================] - 2s 4ms/step - loss: 1220.6710
Epoch 192/200
393/393 [==============================] - 1s 4ms/step - loss: 1209.4661
Epoch 193/200
393/393 [==============================] - 1s 4ms/step - loss: 1204.4553
Epoch 194/200
393/393 [==============================] - 2s 5ms/step - loss: 1185.9669
Epoch 195/200
393/393 [==============================] - 2s 4ms/step - loss: 1184.0776
Epoch 196/200
393/393 [==============================] - 2s 6ms/step - loss: 1166.1237
Epoch 197/200
393/393 [==============================] - 2s 5ms/step - loss: 1174.3926
Epoch 198/200
393/393 [==============================] - 

393/393 [==============================] - 1s 2ms/step - loss: 19858.4395
Epoch 172/200
393/393 [==============================] - 1s 2ms/step - loss: 19710.1621
Epoch 173/200
393/393 [==============================] - 1s 3ms/step - loss: 19512.5664
Epoch 174/200
393/393 [==============================] - 1s 3ms/step - loss: 19239.1191
Epoch 175/200
393/393 [==============================] - 1s 3ms/step - loss: 19032.7109
Epoch 176/200
393/393 [==============================] - 1s 2ms/step - loss: 18826.9883
Epoch 177/200
393/393 [==============================] - 1s 2ms/step - loss: 18542.6836
Epoch 178/200
393/393 [==============================] - 1s 2ms/step - loss: 18414.1836
Epoch 179/200
393/393 [==============================] - 1s 2ms/step - loss: 18251.1621
Epoch 180/200
393/393 [==============================] - 1s 2ms/step - loss: 17978.8789
Epoch 181/200
393/393 [==============================] - 1s 3ms/step - loss: 17743.1211
Epoch 182/200
393/393 [=======================

In [62]:
model_outputs_nn = pd.DataFrame(model_outputs_nn)
model_outputs_nn

,MLP,LSTM,RNN
R-squared,0.996827,0.988807,0.852240
MSE,289.755888,1022.117177,13493.418895
RMSE,17.022217,31.970567,116.161176
MAE,4.494267,9.041862,74.723188


In [63]:
sample_predicted_nn

{'MLP': array([ 114.02331799,  203.77214282, 1036.34164395, 1307.08928   ,
         673.25235903,  250.00021774,  305.31845559,  186.06101643,
         272.51099766,  304.59565055,  128.30556064]),
 'LSTM': array([[ 33.764343],
        [ 13.440798],
        [319.10892 ],
        [509.40875 ],
        [319.83093 ],
        [421.9389  ],
        [348.9947  ],
        [379.78265 ],
        [429.5772  ],
        [395.2431  ],
        [ 35.919037]], dtype=float32),
 'RNN': array([[-579.7477 ],
        [-583.7301 ],
        [ 773.9526 ],
        [ 773.9526 ],
        [ 773.9526 ],
        [ 775.93274],
        [ 773.9526 ],
        [ 872.5875 ],
        [ 788.4404 ],
        [ 841.5123 ],
        [-677.8798 ]], dtype=float32)}

In [64]:
sample_predicted_nn['LSTM'] = sample_predicted_nn['LSTM'].flatten()
sample_predicted_nn['RNN'] = sample_predicted_nn['RNN'].flatten()

In [66]:
sample_predicted_nn = pd.DataFrame(sample_predicted_nn)
real_pred_nn=pd.concat([a[['prıce']],sample_predicted_nn], axis=1)
real_pred_nn

,prıce,MLP,LSTM,RNN
0,116.47,114.023318,33.764343,-579.747681
1,203.02,203.772143,13.440798,-583.730103
2,1038.15,1036.341644,319.108917,773.952576
3,1307.82,1307.089280,509.408752,773.952576
4,674.56,673.252359,319.830933,773.952576
5,243.50,250.000218,421.938904,775.932739
6,303.87,305.318456,348.994690,773.952576
7,184.08,186.061016,379.782654,872.587524
8,275.78,272.510998,429.577209,788.440430
9,327.88,304.595651,395.243103,841.512329
